# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=bert-base-uncased", "case.data.task=masked-lm",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [3]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [16] # This is the sequence length

cfg.case.model = "bert-sanity-check" #  "huawei-noah/TinyBERT_General_4L_312D"
cfg.case.data.tokenizer = "bert-base-uncased"
cfg.case.data.vocab_size = 30522
cfg.case.data.disable_mlm=True


cfg.case.server.provide_public_buffers = True
cfg.case.server.has_external_data = True
cfg.case.server.pretrained = False



cfg.case.server.param_modification.v_length = 64
cfg.case.server.param_modification.eps = 1e-6
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 100000000
cfg.case.server.param_modification.sequence_token_weight = 1
cfg.case.server.param_modification.measurement_scale = 1


cfg.attack.token_strategy="mixed" # "decoder-bias" is only nice for bert if disable_mlm=True

### Instantiate all parties

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Model architecture bert-sanity-check loaded with 109,514,298 parameters and 1,024 buffers.
Overall this is a data ratio of 6844644:1 for target shape [1, 16] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 1

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: bert-sanity-check
        model state: default
        public bu

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Found attention of shape torch.Size([768, 768]).
Computing feature distribution before the probe layer Linear(in_features=768, out_features=3072, bias=True) from external data.
Feature mean is -0.03702928125858307, feature std is 1.2719128131866455.
Computing user update in model mode: eval.


In [6]:
user.print(true_user_data)

[CLS] the tower building of the little rock arsenal, also known as u. s


# Reconstruct user data

In [7]:
# attacker.cfg.sentence_algorithm = "k-means"

In [8]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

Recovered tokens tensor([  101,  1010,  1012,  1055,  1057,  1996,  1997,  2004,  2036,  2124,
         2210,  2311,  2600,  3578,  9433, 27781, 30494]) through strategy mixed.
Recovered 18 embeddings with positional data from imprinted layer.
Reduced to 16 hits.
METRICS: | Accuracy: 0.8750 | S-BLEU: 0.67 | FMSE: 2.4733e-03 | 
 G-BLEU: 0.60 | ROUGE1: 0.89| ROUGE2: 0.72 | ROUGE-L: 0.89| Token Acc: 87.50% | Label Acc: 87.50%
[CLS] the tower門 of antibody little rock arsenal, also known as u. s
